# The Data
July 12, 2022 - new train/valid IDs

In [1]:
import numpy as np
import pandas as pd


In [2]:
FILEPATH='/home/jrm/Adjeroh/Naved/July_Run/TrainTestSplit_82k/'
TRAIN_FILES=['fold0_train.txt','fold1_train.txt','fold2_train.txt','fold3_train.txt','fold4_train.txt']
TEST_FILES=['fold0_test.txt','fold1_test.txt','fold2_test.txt','fold3_test.txt','fold4_test.txt']

In [13]:
def load_df(train,fold):
    filepath = FILEPATH
    if train:
        filepath += TRAIN_FILES[fold]
    else:
        filepath += TEST_FILES[fold]
    df         = pd.read_csv(filepath,header=None)
    df.columns = ['PNG','CANCER']     
    df['CASE'] = df['PNG'].str[0:19]
    df['WSI']  = df['PNG'].str[0:23]
    return df

In [37]:
def triple(df1,df2,column):
    a = df1[column].nunique()
    b = df2[column].nunique()
    return (a, b, a+b)
def basic_stats(train,test):
    print('Category:    %6s\t%5s\t%6s'%('train','test','total'))
    print('Unique CASE: %6d\t%5d\t%6d' % triple(train,test,'CASE'))
    print('Unique WSI:  %6d\t%5d\t%6d' % triple(train,test,'WSI'))
    print('Unique PNG:  %6d\t%5d\t%6d' % triple(train,test,'PNG'))
    a = train['CANCER'].value_counts()
    b = test['CANCER'].value_counts()
    for cancer in range(6):
        aa = a[cancer]
        bb = b[cancer]
        print('Class %d:     %6d\t%5d\t%6d' % (cancer,aa,bb,aa+bb))


In [38]:
for fold in range(5):
    print('Fold',fold)
    train_df = load_df(True,fold)
    test_df = load_df(False,fold)
    basic_stats(train_df,test_df)

Fold 0
Category:     train	 test	 total
Unique CASE:     83	   25	   108
Unique WSI:     171	   45	   216
Unique PNG:   65600	17370	 82970
Class 0:      38865	 6101	 44966
Class 1:       9391	 3511	 12902
Class 2:       9273	 4677	 13950
Class 3:       4873	 1491	  6364
Class 4:       2003	 1194	  3197
Class 5:       1195	  396	  1591
Fold 1
Category:     train	 test	 total
Unique CASE:     83	   25	   108
Unique WSI:     160	   56	   216
Unique PNG:   61605	21365	 82970
Class 0:      32935	12031	 44966
Class 1:       8983	 3919	 12902
Class 2:      11108	 2842	 13950
Class 3:       5000	 1364	  6364
Class 4:       2393	  804	  3197
Class 5:       1186	  405	  1591
Fold 2
Category:     train	 test	 total
Unique CASE:     83	   25	   108
Unique WSI:     168	   48	   216
Unique PNG:   65500	17470	 82970
Class 0:      39186	 5780	 44966
Class 1:      10166	 2736	 12902
Class 2:       7946	 6004	 13950
Class 3:       4608	 1756	  6364
Class 4:       2399	  798	  3197
Class 5:       1195	  